# Подготовка изображений и тестирование модели TF Lite

In [13]:
!pip install keras_image_helper

In [14]:
from keras_image_helper import create_preprocessor

In [15]:
# Создаем функцию-препроцессинга для последующего преобразования в массив Nympy

preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [16]:
image_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRa_4rRYSPhQS5RlTPshFtALG2l3q6O-dVOhw&s'
X = preprocessor.from_url(image_url)

In [17]:
X.shape

(1, 299, 299, 3)

- 1 - количество изображений
- 299 - высота
- 299 - ширина
- 3 - количество цветовых каналов: красный, зеленый и синий

In [18]:
import tensorflow.lite as tflite

Загружаем ранее преобразованную модель:

In [19]:
interpreter = tflite.Interpreter(model_path='Deployment/clothing-model-v4-0.886.tflite') # создает интерпретатор
interpreter.allocate_tensors() # инициализирует интерпретатор с помощью модели

Получаем входные данные (куда пойдет Х) и выходные данные (откуда получим прогнозы)

In [20]:
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']

output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

Делаем предсказания:

In [21]:
interpreter.set_tensor(input_index, X) # Помещает Х во входные данные
interpreter.invoke() # Запускает модель для получения прогнозов

preds = interpreter.get_tensor(output_index) # Получает прогнозы

Переменная preds содержит вероятности принадлежности к каждому классу:

In [22]:
preds

array([[-3.649766  , -5.0966783 , -2.2770104 , -0.53570926,  8.526883  ,
        -1.2590158 , -2.098923  ,  3.7655292 , -2.1019533 , -3.2585163 ]],
      dtype=float32)

Конвертируем предсказания в читаемый формат:

In [23]:
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [24]:
results = dict(zip(labels, preds[0]))
results

{'dress': np.float32(-3.649766),
 'hat': np.float32(-5.0966783),
 'longsleeve': np.float32(-2.2770104),
 'outwear': np.float32(-0.53570926),
 'pants': np.float32(8.526883),
 'shirt': np.float32(-1.2590158),
 'shoes': np.float32(-2.098923),
 'shorts': np.float32(3.7655292),
 'skirt': np.float32(-2.1019533),
 't-shirt': np.float32(-3.2585163)}

Метка pants имеет наибольшую вероятность, что означает, что модель считает, что на изображении изображены брюки.